# Evaluation & Results

**Paper Metric**: Deferral curves showing cost vs accuracy tradeoff

**Key Result**: Universal routing works with new models without retraining

In [ ]:
# Setup and imports
import os
import json
import pickle
import numpy as np
from typing import Literal
from sentence_transformers import SentenceTransformer
from pydantic import BaseModel
import matplotlib.pyplot as plt
from datasets import load_dataset

# Load saved models and data
with open('/content/universal_kmeans.pkl', 'rb') as f:
    kmeans = pickle.load(f)

with open('/content/model_database.json', 'r') as f:
    model_db = json.load(f)

# Convert lists back to numpy arrays
for model_name in model_db:
    model_db[model_name]['psi_vector'] = np.array(model_db[model_name]['psi_vector'])

# Initialize embedder
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# API keys
API_KEYS = {
    'openai': os.getenv('OPENAI_API_KEY', 'api_key'),
    'groq': os.getenv('GROQ_API_KEY', 'api_key')
}

print(f"✅ Loaded {len(model_db)} models from database")
print(f"✅ Loaded K-means model with {kmeans.n_clusters} clusters")

## Test Set Evaluation

Independent test set (different from validation used for characterization)

In [ ]:
# Define helper functions
class MCQAnswer(BaseModel):
    answer: Literal["A", "B", "C", "D"]

def call_llm(model_name: str, provider: str, prompt: str) -> str:
    """Call LLM and extract A/B/C/D answer"""
    try:
        if provider == 'openai':
            from openai import OpenAI
            client = OpenAI(api_key=API_KEYS['openai'])

            if 'gpt-4' in model_name:
                response = client.beta.chat.completions.parse(
                    model=model_name,
                    messages=[{"role": "user", "content": prompt}],
                    response_format=MCQAnswer,
                    temperature=0
                )
                return response.choices[0].message.parsed.answer

        elif provider == 'groq':
            from groq import Groq
            client = Groq(api_key=API_KEYS['groq'])

            response = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt + " Answer with only A, B, C, or D."}],
                temperature=0,
                max_tokens=5
            )

            answer = response.choices[0].message.content.strip().upper()
            for letter in ["A", "B", "C", "D"]:
                if letter in answer:
                    return letter

    except Exception as e:
        print(f"Error with {model_name}: {e}")

    return "E"  # Error

# Define UniRouter class
class UniRouter:
    def __init__(self, model_db, kmeans_model, embedder):
        self.model_db = model_db
        self.kmeans = kmeans_model
        self.embedder = embedder
        self._normalize_costs()

    def _normalize_costs(self):
        costs = [info['cost'] for info in self.model_db.values()]
        self.min_cost, self.max_cost = min(costs), max(costs)

        for info in self.model_db.values():
            info['norm_cost'] = (info['cost'] - self.min_cost) / (self.max_cost - self.min_cost)

    def route(self, prompt: str, lambda_cost: float = 0.1) -> dict:
        """Route prompt using paper's algorithm"""

        # Find cluster
        embedding = self.embedder.encode([prompt])[0]
        cluster_id = self.kmeans.predict(embedding.reshape(1, -1))[0]

        # Score models: error + λ × cost
        best_model, best_score = None, float('inf')

        for model_name, info in self.model_db.items():
            error_rate = info['psi_vector'][cluster_id]
            score = error_rate + lambda_cost * info['norm_cost']

            if score < best_score:
                best_score, best_model = score, model_name

        return {
            'model': best_model,
            'cluster': int(cluster_id),
            'cost': self.model_db[best_model]['cost']
        }

# Initialize router
router = UniRouter(model_db, kmeans, embedder)
print("✅ Router initialized")

In [ ]:
# Create test set
test_dataset = load_dataset("cais/mmlu", "all", split="test")
test_dataset = test_dataset.shuffle(seed=123).select(range(50))

test_set = []
for item in test_dataset:
    prompt = f"{item['question']}\nA) {item['choices'][0]}\nB) {item['choices'][1]}\nC) {item['choices'][2]}\nD) {item['choices'][3]}\nAnswer:"
    test_set.append({
        'prompt': prompt,
        'answer': ['A', 'B', 'C', 'D'][item['answer']]
    })

print(f"✅ Test set: {len(test_set)} examples")

## Deferral Curve Analysis

**Core Experiment**: How does accuracy change with cost as we vary λ?

In [ ]:
lambda_values = [0.001, 0.01, 0.1, 1.0, 10.0]
results = []

for λ in lambda_values:
    correct, total_cost = 0, 0

    for example in test_set:
        # Route and predict
        routing = router.route(example['prompt'], lambda_cost=λ)
        selected = routing['model']
        total_cost += routing['cost']

        response = call_llm(
            selected,
            router.model_db[selected]['provider'],
            example['prompt']
        )

        if response == example['answer']:
            correct += 1

    accuracy = correct / len(test_set)
    avg_cost = total_cost / len(test_set)

    results.append({'lambda': λ, 'accuracy': accuracy, 'cost': avg_cost})
    print(f"λ={λ:5.3f}: {accuracy:.1%} accuracy, ${avg_cost:.3f} avg cost")

## Deferral Curve Visualization

In [ ]:
costs = [r['cost'] for r in results]
accuracies = [r['accuracy'] for r in results]

plt.figure(figsize=(10, 6))
plt.plot(costs, accuracies, 'bo-', linewidth=2, markersize=8)

for r in results:
    plt.annotate(f"λ={r['lambda']}", (r['cost'], r['accuracy']),
                xytext=(5, 5), textcoords='offset points')

plt.xlabel('Average Cost ($)')
plt.ylabel('Accuracy')
plt.title('UniRoute Deferral Curve\n(Jitkrittum et al., 2025)')
plt.grid(True, alpha=0.3)
plt.show()

print("\n✅ Experiment complete!")

In [ ]:
# Function to characterize new models
from collections import defaultdict

def characterize_model(model_name: str, provider: str) -> np.ndarray:
    """Compute Ψ(m) error profile for a new model"""
    print(f"Characterizing {model_name}...")

    # Load validation set to compute error rates
    with open('data/universal_validation_set.json', 'r') as f:
        validation_data = json.load(f)

    # Group validation data by cluster
    validation_clusters = defaultdict(list)
    for item in validation_data:
        validation_clusters[item['cluster']].append(item)

    K = kmeans.n_clusters
    psi_vector = np.zeros(K)

    for cluster_id in range(K):
        cluster_data = validation_clusters[cluster_id]
        errors = 0

        for example in cluster_data:
            response = call_llm(model_name, provider, example['prompt'])
            if response != example['answer']:
                errors += 1

        psi_vector[cluster_id] = errors / len(cluster_data) if cluster_data else 0
        print(f"  Cluster {cluster_id}: {psi_vector[cluster_id]:.1%}")

    return psi_vector

## Adding New Models (Zero Retraining)

**Paper's Key Advantage**: Add new models by just computing their Ψ(m) profile

In [ ]:
def add_new_model(model_name: str, provider: str, cost: float):
    """Add model without retraining router"""

    # Compute error profile on existing clusters
    psi_vector = characterize_model(model_name, provider)

    # Add to database
    router.model_db[model_name] = {
        'psi_vector': psi_vector,
        'provider': provider,
        'cost': cost
    }

    router._normalize_costs()
    print(f"✅ {model_name} added!")

# Example: Add better model
# add_new_model('llama-3.3-70b-versatile', 'groq', 0.69)

print("💡 New models can be added dynamically!")

## Key Insights (Jitkrittum et al., 2025)

- ✅ **Universal**: Works with any new LLM
- ✅ **Efficient**: No router retraining needed
- ✅ **Practical**: Real cost-quality optimization
- ✅ **Scalable**: O(K) cost to add new models